In [13]:
###  Buiding agentic chatbot with langgraph
from typing import Annotated
from  typing_extensions import TypedDict
from langgraph.graph import StateGraph,StateGraph,START,END
from langgraph.graph.message import add_messages

In [14]:
class State(TypedDict):
    messages:Annotated[list,add_messages]
    
graph_builder=StateGraph(State)

In [15]:
graph_builder

In [18]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
from langchain_groq import ChatGroq
from langchain.chat_models import  init_chat_model

llm=ChatGroq(model="llama-3.1-8b-instant")

In [20]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001707D818560>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001707C3A1A60>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [21]:
### Nide functionality
def chatbot(state:State):
    return {"messages":[llm.invoke(state["messges"])]}

In [ ]:
graph_builder=StateGraph(State)